## 1) Process VIN data

In [2]:
import vin_processing as vp
from datetime import datetime
import pandas as pd
reload(vp)

Rebuilding cache of generated files for COM support...
Checking 00020813-0000-0000-C000-000000000046x0x1x8
Could not add module (IID('{00020813-0000-0000-C000-000000000046}'), 0, 1, 8) - <type 'exceptions.ImportError'>: cannot import name Dispatch
Done.


<module 'vin_processing' from 'vin_processing.pyc'>

### 1) Read the VINs from both files (txsafe18 and forybvin), turn them into a list and then feed that to the function that makes use of the web API.

In [35]:
file_path = '../data/txsafe18.txt'
counts_with_vins = vp.get_counts(file_path=file_path, put_in_db=False)
vin_list = counts_with_vins.VIN.tolist()
vin_list = [v for v in vin_list if len(v) >= 17]
len(vin_list)

244003

In [34]:
other_file_path = '../data/forybvin.txt'
other_counts_with_vins = vp.get_counts(file_path=other_file_path, put_in_db=False)
other_vin_list = other_counts_with_vins.VIN.tolist()
other_vin_list = [v for v in other_vin_list if len(v) >= 17]
len(other_vin_list)

137382

In [38]:
vin_list = list(set(vin_list + other_vin_list))
len(vin_list)

370920

### 2) Run the VIN decoder untll either 1) there are no more missing VINs (all VINs have been decoded), or 2) `max_iter` has been reached.

In [39]:
reload(vp)
file_name = r'out_parallel'
if_exists = 'replace'
i = 0
max_iter = 5
missing_vins = vin_list
vin_df = pd.DataFrame()
while missing_vins and (i < max_iter):
    print 'Loop {}...'.format(i)
    vin_file_path = r'X:\EPA_MPG\data\{}.csv'.format(file_name + str(i))
    vp.get_data_parallel_stream(missing_vins, vin_file_path=vin_file_path)
    print '\tFixing column names'
    vp.take_out_results_string(vin_file_path)
    print '\tAdd vtyp and concat dataframes'
    vin_df = pd.concat([vp.add_vtyp(vin_file_path=vin_file_path, if_exists=if_exists), vin_df], axis=0)    
    missing_vins = list(set(vin_list) - set(vin_df.VIN))
    print '\tWe\'re still missing {} VINs {}'.format(len(missing_vins), missing_vins)
    i += 1
    if_exists = 'append'

Loop 0...


Done 100.000%

	Fixing column names
	Add vtyp and concat dataframes
	We're still missing VINs ['25289660831800605', '47795200612640477', '25489105858002548', '22027509821401317', '11111111111111111', '20080500000005394', '13915122761304112', '58761243684633995', '1XPCD68XXTN408520']
Loop 1...


Done 100.000%

	Fixing column names
	Add vtyp and concat dataframes
	We're still missing VINs ['25289660831800605', '47795200612640477', '25489105858002548', '22027509821401317', '11111111111111111', '20080500000005394', '13915122761304112', '58761243684633995']
Loop 2...


...

	Fixing column names
	Add vtyp and concat dataframes
	We're still missing VINs ['25289660831800605', '47795200612640477', '25489105858002548', '22027509821401317', '11111111111111111', '20080500000005394', '13915122761304112', '58761243684633995']
Loop 8...


Done 100.000%

	Fixing column names
	Add vtyp and concat dataframes
	We're still missing VINs ['25289660831800605', '47795200612640477', '25489105858002548', '22027509821401317', '11111111111111111', '20080500000005394', '13915122761304112', '58761243684633995']
Loop 9...


Done 100.000%

	Fixing column names
	Add vtyp and concat dataframes
	We're still missing VINs ['25289660831800605', '47795200612640477', '25489105858002548', '22027509821401317', '11111111111111111', '20080500000005394', '13915122761304112', '58761243684633995']


In [3]:
# compare vin_df to what's in the database
vin_df_from_db = pd.read_sql('vin_with_vtyp_no_counts', vp.engine)
print(vin_df_from_db.shape)

(370992, 149)


In [6]:
len(vin_df_from_db.VIN.unique())

370920

In [5]:
vp.add_counts()

## Merge

In [7]:
%run merge
# Load and modify the basics
vin_original, epa_original = load()

In [8]:
vin_original = vin_original.loc[vin_original.VIN.apply(len) >= 17]

In [9]:
%run merge
vin_original_fixed, epa_original_fixed = fix(vin_original, epa_original, init_yr=0)

In [52]:
%run merge
vin_expanded, epa_expanded = split_and_expand(vin_original_fixed, epa_original_fixed)

Expanding VIN data
Expanding EPA data
Making last minute changes


In [115]:
%run merge
vin, epa = modify(vin_expanded, epa_expanded)

In [117]:
%run merge
t0 = datetime.now()
matched_vins, vin_merge = run_merge(vin, epa)    
t1 = datetime.now()
print '\nRuntime: {}'.format(t1-t0)

Merging datasets
**************************************************
('Merging using:', ['make', 'model_mod', 'year', 'fuelType1_mod'])
('Matching on ', ['make', 'model_mod', 'year', 'fuelType1_mod', 'type', 'drive_mod', 'displ_mod', 'cylinders', 'transmission_type_mod', 'transmission_speeds_mod'])
Weighted match fraction: 0.95%
('Matching on ', ['make', 'model_mod', 'year', 'fuelType1_mod', 'type', 'drive_mod', 'displ_mod', 'cylinders', 'transmission_type_mod'])
Weighted match fraction: 1.28%
('Matching on ', ['make', 'model_mod', 'year', 'fuelType1_mod', 'type', 'drive_mod', 'displ_mod', 'cylinders', 'transmission_speeds_mod'])
Weighted match fraction: 1.30%
('Matching on ', ['make', 'model_mod', 'year', 'fuelType1_mod', 'type', 'drive_mod', 'displ_mod', 'transmission_type_mod', 'transmission_speeds_mod'])
Weighted match fraction: 1.42%
('Matching on ', ['make', 'model_mod', 'year', 'fuelType1_mod', 'type', 'drive_mod', 'cylinders', 'transmission_type_mod', 'transmission_speeds_mod'])

Weighted match fraction: 94.42%
('Matching on ', ['make', 'model_mod', 'year', 'type', 'displ_mod', 'cylinders', 'transmission_type_mod', 'transmission_speeds_mod'])
Weighted match fraction: 94.43%
('Matching on ', ['make', 'model_mod', 'year', 'drive_mod', 'displ_mod', 'cylinders', 'transmission_type_mod', 'transmission_speeds_mod'])
Weighted match fraction: 94.43%
('Matching on ', ['make', 'model_mod', 'year', 'type', 'drive_mod', 'displ_mod', 'cylinders'])
Weighted match fraction: 95.15%
('Matching on ', ['make', 'model_mod', 'year', 'type', 'drive_mod', 'displ_mod', 'transmission_type_mod'])
Weighted match fraction: 95.15%
('Matching on ', ['make', 'model_mod', 'year', 'type', 'drive_mod', 'displ_mod', 'transmission_speeds_mod'])
Weighted match fraction: 95.15%
('Matching on ', ['make', 'model_mod', 'year', 'type', 'drive_mod', 'cylinders', 'transmission_type_mod'])
Weighted match fraction: 95.15%
('Matching on ', ['make', 'model_mod', 'year', 'type', 'drive_mod', 'cylinders', 'tra

In [129]:
%run merge
dfs = output(vin, epa, matched_vins, export=True, export_all=True)

Merge fraction weighted: 98.87%
Duplication rate: 2.564
0.0     66013
1.0      9862
2.0      8499
3.0      4562
4.0      1991
5.0      1126
6.0       612
7.0       233
8.0       143
9.0        93
10.0       13
11.0       23
12.0       14
14.0       13
15.0       14
16.0       32
17.0       25
31.0       13
34.0        8
Name: highway08_spread, dtype: int64
0.0     70.761826
1.0     10.571450
2.0      9.110399
3.0      4.890180
4.0      2.134228
5.0      1.207002
6.0      0.656026
7.0      0.249761
8.0      0.153287
9.0      0.099690
10.0     0.013935
11.0     0.024655
12.0     0.015007
14.0     0.013935
15.0     0.015007
16.0     0.034302
17.0     0.026798
31.0     0.013935
34.0     0.008576
Name: highway08_spread, dtype: float64
0.0     67715
1.0     13339
2.0      7728
3.0      2445
4.0      1063
5.0       460
6.0       230
7.0       130
8.0         4
9.0        23
10.0       22
11.0        7
12.0       33
15.0       27
16.0       23
17.0        1
18.0       11
19.0        7
40.0    

## Debugging

In [102]:
vin, epa = vin_expanded.copy(), epa_expanded.copy()

# Make the counts integers. 
vin['counts'] = vin['counts'].astype(int)

# Modify transmission information
## In vin DB: turn transmission speeds into integers then strings.
vin['transmission_speeds_mod'] = vin['transmission_speeds'].apply(lambda s: unicode(try_int(s)))
## In epa DB: transform info in epa database to get trammission speeds and types.
## Transmission speeds.
def get_transmission_speeds(s):
    try:
        return re.search(r'\d+', s).group()
    except:
        return None
## Transmission type.
def get_transmission_type(s):
    if isinstance(s, unicode):
        if re.search(r'auto', s):
            return "auto"
        else:
            return "manu"
## Apply to epa.
epa['transmission_speeds_mod'] = epa['transmission_speeds'] = epa.trany.apply(get_transmission_speeds)
epa['transmission_type_mod'] = epa['transmission_type'] = epa.trany.apply(get_transmission_type)

# Round displacement in both databases.
def convert_displacement(s):
    if re.findall(',', s):
        return unicode(round(float(s.split(',')[0]), 1))
    elif s == u'-1':
        return s
    else:
        return unicode(round(float(s), 1))
for df in (epa, vin):
    df['displ_mod'] = df['displ'].apply(convert_displacement)

# Change type of mpg values to be floats. 
mpg_list = 'highway08, highway08U, comb08, comb08U, city08, city08U'.split(', ')
epa[mpg_list] = epa[mpg_list].astype(float)

# Add vtyp3 for epa. 
epa['vtyp3'] = u'-1'
epa.loc[epa.VClass.str.contains('pickup') & epa.model.str.contains('15|10'), 'vtyp3'] = u'3'
epa.loc[epa.VClass.str.contains('pickup') & epa.model.str.contains('25|35'), 'vtyp3'] = u'4'

# Add weight variable. 
weight_df = vin.GVWR.str.extract(r'([\d,]*) lb').fillna('0')
if pd.__version__ <= u'0.22.0':
    weight_series = weight_df
else:
    weight_series = weight_df[0]
vin['weight'] = weight_series.str.replace(',', '').astype(int)

# Drop heavy vehicles. 
weight_limit = 9000
vin = vin.loc[vin.weight < weight_limit]

# Modify makes.
vin, epa = mod_makes(vin, epa)

In [95]:
len(epa.loc[(epa.make == 'lexus') & (epa.model_mod.str.contains(r'(?:gs|sc)\d+')), 
        'model_mod'].str.extract(r'(\D+)\d+'))

81

In [96]:
len(epa.loc[(epa.make == 'lexus') & (epa.model_mod.str.contains(r'(?:gs|sc)\d+')), 'model_mod'])

81

In [103]:
epa.loc[(epa.make == 'lexus') & (epa.model_mod.str.contains(r'(?:gs|sc)\d+')), 'model_mod'] = \
    epa.loc[(epa.make == 'lexus') & (epa.model_mod.str.contains(r'(?:gs|sc)\d+')), 
        'model_mod'].str.extract(r'(\D+)\d+').values


In [88]:
any(vin.model_mod.isna())

False

In [104]:
any(epa.model_mod.isna())

False

In [69]:
# For Lexus models, drop the numbers. 
# Get rid of 'new' from all volkswagen models. 
vin.loc[(vin.make == 'volkswagen') & (vin.model_mod.str.contains('new')), 'model_mod'] = \
    vin.loc[(vin.make == 'volkswagen') & (vin.model_mod.str.contains('new')), 'model_mod'].str.extract('new(.*)')
epa.loc[(epa.make == 'volkswagen') & (epa.model_mod.str.contains('new')), 'model_mod'] = \
    epa.loc[(epa.make == 'volkswagen') & (epa.model_mod.str.contains('new')), 'model_mod'].str.extract('new(.*)')
# In EPA, change the model name to crosstour when the model name contains that string. 
epa.loc[(epa.make == 'honda') & (epa.model.str.contains('crosstour')), 'model_mod'] = 'crosstour'
# EPA has a separate model called accord wagon; in VIN, BodyClass identifies the wagons; 
# so we create a separate model in VIN called accord-wagon.
vin.loc[(vin.make == 'honda') & (vin.model == 'accord') & (vin.BodyClass == 'wagon'), 'model_mod'] = 'accord-wagon'
epa.loc[(epa.make == 'honda') & (epa.model == 'accord wagon'), 'model_mod'] = 'accord-wagon'
# Honda HX. 
epa.loc[(epa.model.str.contains('(?=(?:.*civic.*))(?=(?:.*hx.*))')) & (epa.make == 'honda'), 
    'model_mod'] = 'civichx'
vin.loc[(vin.Series.str.contains('hx')) & (vin.model.str.contains('civic')) & (vin.make == 'honda'), 
    'model_mod'] = 'civichx'
# EPA has a separate model called matrix, whereas VIN has a model corolla matrix; 
# so we create a separate model in VIN called matrix.
vin.loc[(vin.make == 'toyota') & (vin.model == 'corolla matrix'), 'model_mod'] = 'matrix'
## Modify acura models. 
epa.loc[(epa.make == 'acura') & epa.model.str.contains(r'\d\.\d.*'), 'model_mod'] = \
    epa.loc[(epa.make == 'acura') & epa.model.str.contains(r'\d\.\d.*'), 'model_mod'].apply(
        lambda s: re.sub(r'\d\.\d(.*)', r'\1', s))
## Infiniti. 
epa.loc[(epa.make == 'infiniti') & epa.model.str.contains(r'(?=.*?)x(?=$|\s)'), 'model_mod'] = \
    epa.loc[(epa.make == 'infiniti') & epa.model.str.contains(r'(?=.*?)x(?=$|\s)'), 'model_mod'].apply(
        lambda s: re.sub(r'(.*?)x(?=$|\s).*', r'\1', s))
## Nissan. 
epa.loc[(epa.make == 'nissan'), 'model_mod'] = \
    epa.loc[(epa.make == 'nissan'), 'model_mod'].replace('truck', 'pickup', regex=True)
epa.loc[epa.model.str.contains('pathfinder armada'), 'model_mod']	= 'armada'
## BMW. 
epa.loc[(epa.make == 'bmw'), 'model_mod'] = epa.loc[(epa.make == 'bmw'), 'model_mod'].apply(lambda s: s[0])
vin.loc[(vin.make == 'bmw'), 'model_mod'] = vin.loc[(vin.make == 'bmw'), 'model_mod'].apply(lambda s: s[0])
## Delete from EPA all models that contain chassis in model name. 
epa = epa.loc[~epa.model.str.contains('chassis')]
## All models with displacements in the model name, e.g. '190e 2.3-16'
def mod_models_w_displ(s):
    pattern = re.compile(r'(.*)\d\.\d(.*)')
    groups = re.search(pattern, s).groups()
    return groups[0].strip() or groups[1].strip()
## Lincoln.
### Replace 'zephyr' with 'mkz' for VIN for make 'lincoln'
vin.loc[(vin.make == 'lincoln') & (vin.model.str.contains('zephyr')), 'model_mod'] = 'mkz'
## Ford.
### Replace 'ltd crown victoria' with 'crown victoria' in EPA for make 'ford'
epa.loc[(epa.make == 'ford') & (epa.model.str.contains('ltd crown victoria')), 'model_mod'] = 'crown victoria'
### Replace 'crown victoria' with 'crown victoria police' in VIN for make 'ford' if `series = 'police interceptor'`
vin.loc[(vin.make == 'ford') & (vin.model.str.contains('crown victoria')) & (vin.Series == 'police interceptor'),
    'model_mod'] = 'crown victoria police'
## Chrysler. 
### Replace '300c' and '300c/srt' with '300' in EPA
epa.loc[(epa.make == 'chrysler') & (epa.model.str.contains('300')), 'model_mod'] = '300'
## Saturn.
### Replace L100/200 with LS1 in EPA when year is larger than 2001 (inclusive)
epa.loc[(epa.make == 'saturn') & (epa.model == 'l100/200') & (epa.year >= 2001), 'model_mod'] = 'ls1'
vin.loc[(vin.make == 'saturn') & (vin.model_mod != 'ls1'), 'model_mod'] = \
    vin.loc[(vin.make == 'saturn') & (vin.model_mod != 'ls1'), 'model_mod'].apply(
        lambda s: re.search(r'([^\d]+)[\d]', s).groups()[0] if re.search(r'([^\d]+)[\d]', s) else s)
## Mercedes. 
class_index = vin.loc[(vin.make == 'mercedes-benz') & (vin.model.str.contains(r'(?:\D+)-class'))].index
vin.loc[class_index, 'model_mod'] = vin.loc[class_index, 'model'].str.extract(r'(\D+)-class')
digit_class_index = vin.loc[(vin.make == 'mercedes-benz') & (vin.model.str.contains(r'(?:\d+)'))].index
vin.loc[digit_class_index, 'model_mod'] = \
    vin.loc[digit_class_index, 'Series'].str.split(' ').apply(lambda xs: xs[0]).str.extract(r'.*?(\D+)')
vin.loc[(vin.make == 'mercedes-benz') & (vin.model_mod == 'm'), 'model_mod'] = 'ml'
### AMG models.
vin.loc[(vin.make == 'mercedes-benz'), 'model_mod'], epa.loc[(epa.make == 'mercedes-benz'), 'model_mod'] = \
    [df.loc[(df.make == 'mercedes-benz'), 'model_mod'].str.replace('amg', '').str.strip() for df in (vin, epa)]
### Get the models that start with numbers, e.g. 190 sl and 200sel, and keep only the numbers.
index = epa.loc[(epa.make == 'mercedes-benz') & (epa.model.str.contains(r'^\d+')), 'model_mod'].index
epa.loc[index, 'model_mod'] = epa.loc[index, 'model_mod'].str.extract(r'^(\d+)')
### Get the models that start with letters, e.g. sl190 and sel 190, and keep only the letters.
index = epa.loc[(epa.make == 'mercedes-benz') & (epa.model.str.contains(r'^\D+')), 'model_mod'].index
epa.loc[index, 'model_mod'] = epa.loc[index, 'model_mod'].str.extract(r'^(\D+)')
## Toyota. 
### Prius Eco
vin.loc[(vin.VIN.apply(lambda s: s[3:8]) == 'karfu') & (vin.model_mod.str.contains('prius')), 'model_mod'] = 'priuseco'
epa.loc[epa.model.str.contains('prius plug-in'), 'model_mod'] = 'prius plug-in'
pattern = r'c|v|prime|plug-in|eco'
vin.loc[vin.model.str.contains(r'prius (?:{})'.format(pattern)), 'model_mod'] = \
    vin.loc[vin.model.str.contains(r'prius (?:{})'.format(pattern)), 'model_mod'].apply(lambda s: s.replace(' ', ''))
epa.loc[epa.model.str.contains(r'prius (?:{})'.format(pattern)), 'model_mod'] = \
    epa.loc[epa.model.str.contains(r'prius (?:{})'.format(pattern)), 'model_mod'].apply(lambda s: s.replace(' ', ''))
vin.loc[vin.make == 'scion', 'model_mod'] = \
    vin.loc[vin.make == 'scion', 'model_mod'].apply(lambda x: x.split(' ')[1] if len(x.split(' '))>1 else x)
epa.loc[epa.model.str.contains('solara'), 'model_mod'] = 'solara'
vin.loc[vin.model.str.contains('4-runner'), 'model_mod'] = '4runner'
epa.loc[(epa.make == 'toyota'), 'model_mod'] = \
    epa.loc[(epa.make == 'toyota'), 'model_mod'].replace('truck', 'pickup', regex=True)
vin.loc[(vin.make == 'toyota') & (vin.model == 'camry') & (vin.year == 2002) & (vin.displ_mod == '2.2'),
    'displ_mod'] = '2.4'
## Mazda.
def delete_mazda(s):
    match = re.search('mazda(.*)', s)
    if match:
        return match.groups()[0]
    else:
        return s
vin.loc[vin['make'] == 'mazda', 'model_mod'] = \
    vin.loc[vin['make'] == 'mazda', 'model_mod'].apply(delete_mazda)
epa.loc[(epa.make == 'mazda') & (epa.model_mod.str.contains(r'^b\d')), 'model_mod'] = \
    epa.loc[(epa.make == 'mazda') & (epa.model_mod.str.contains(r'^b\d')), 'model_mod'].apply(lambda x: x[0])
### Add displacement and cylinder where it's missing.
vin.loc[(vin['make'] == 'mazda') & vin.model.str.contains('626') & 
    vin.VIN.apply(lambda x: x[7]).isin('c,e'.split(',')), 'displ_mod'] = 2.0
vin.loc[(vin['make'] == 'mazda') & vin.model.str.contains('626') & 
    vin.VIN.apply(lambda x: x[7]).isin('d,f'.split(',')), 'displ_mod'] = 2.5
vin.loc[(vin['make'] == 'mazda') & vin.model.str.contains('protege') & 
    vin.VIN.apply(lambda x: x[6:8]).isin('41'.split(',')), 'displ_mod'] = 1.5
vin.loc[(vin['make'] == 'mazda') & vin.model.str.contains('protege') & 
    vin.VIN.apply(lambda x: x[6:8]).isin('42,21,23'.split(',')), 'displ_mod'] = 1.8
vin.loc[(vin['make'] == 'mazda') & vin.model.str.contains('protege') & 
    vin.VIN.apply(lambda x: x[6:8]).isin('22,24'.split(',')), 'displ_mod'] = 1.6
## Mini. 

ValueError: cannot index with vector containing NA / NaN values

In [107]:
epa.head()

,city08,city08U,comb08,comb08U,cylinders,displ,drive,eng_dscr,fuelType1,highway08,...,EPA_ID,transmission_speeds_mod,transmission_speeds,transmission_type_mod,transmission_type,displ_mod,vtyp3,type,vtyp,vtyp2
0,19.0,0.0,21.0,0.0,4,2.0,rear-wheel drive,(ffs),regular gasoline,25.0,...,1,5,5,manu,manu,2.0,-1,0,car,car
1,9.0,0.0,11.0,0.0,12,4.9,rear-wheel drive,(guzzler),regular gasoline,14.0,...,2,5,5,manu,manu,4.9,-1,0,car,car
2,23.0,0.0,27.0,0.0,4,2.2,front-wheel drive,(ffs),regular gasoline,33.0,...,3,5,5,manu,manu,2.2,-1,0,car,car
3,10.0,0.0,11.0,0.0,8,5.2,rear-wheel drive,-1,regular gasoline,12.0,...,4,3,3,auto,auto,5.2,-1,0,van,van
4,10.0,0.0,11.0,0.0,8,5.2,rear-wheel drive,-1,regular gasoline,12.0,...,5,3,3,auto,auto,5.2,-1,0,van,van


In [111]:
vin_original.loc[(vin_original.Model != None) & (vin_original.Model.str.lower().str.contains('lumina')), 'Model'].unique()

array([u'Lumina APV', u'Lumina'], dtype=object)

In [106]:
epa_original_fixed.loc[(epa_original_fixed.model.str.contains('lumina')), 'model'].unique()

array([u'lumina', u'lumina/apv minivan 2wd', u'lumina minivan 2wd',
       u'lumina/monte carlo'], dtype=object)

In [195]:
	vin_original.loc[(vin_original.year == 2011) & (vin_original.make.str.lower() == 'chevrolet') & 
		(vin_original.model.str.lxower().str.contains('silverado') & (vin_original.fuelType1_mod == 'diesel')), 
		['cylinders', 'displ']]


,cylinders,displ
67561,8,6.6
67600,8,6.6
67626,8,6.6
67662,8,6.6
67663,8,6.6
67710,8,6.6
67712,8,6.6
67726,8,6.6
67736,8,6.6
67743,8,6.6


In [203]:
sorted(list(vin.columns.str.lower()))

[u'bodyclass',
 u'counts',
 'cylinders',
 'displ',
 'displ_mod',
 'drive',
 'drive_mod',
 'error_id',
 'fueltype1',
 'fueltype1_mod',
 'fueltype2',
 u'gvwr',
 'make',
 'model',
 'model_mod',
 u'series',
 'transmission_speeds',
 'transmission_speeds_mod',
 'transmission_type',
 'transmission_type_mod',
 u'trim',
 u'trim2',
 'type',
 'type_from_vin',
 u'vehicletype',
 u'vin',
 'vin_id',
 u'vtyp3',
 'weight',
 'year']

In [189]:
	vin_original.loc[(vin_original.year == 2011) & (vin_original.make.str.lower() == 'chevrolet') & 
		(vin_original.model.str.lower().str.contains('silverado') & (vin_original.fuelType1_mod == 'diesel')), 
		['cylinders', 'displ']]

In [182]:
vin_original.loc[(vin_original.year == 2006) & (vin_original.make.str.lower() == 'chevrolet') & 
		(vin_original.model.str.lower().str.contains('tahoe|suburban') & (vin_original.fuelType1_mod == 'hev')), 'model'].unique()

array([u'tahoe', u'suburban'], dtype=object)

In [165]:
vin_original.model.unique()

array([u'truck', u'oshkosh', u'oshkosh trailer', ..., u'camry hybrid',
       u'venza', u'smith electric vehicles'], dtype=object)

In [157]:
vin_original.columns

Index([            u'BodyClass',                 u'displ',
                       u'drive',             u'cylinders',
                   u'fuelType1',             u'fuelType2',
                        u'GVWR',                  u'make',
                       u'model',                  u'year',
                      u'Series',   u'transmission_speeds',
           u'transmission_type',                  u'Trim',
                       u'Trim2',                   u'VIN',
                 u'VehicleType',                u'counts',
                       u'vtyp3',              u'error_id',
       u'transmission_type_mod',         u'fuelType1_mod',
                   u'drive_mod'],
      dtype='object')

In [ ]:
test

In [407]:
sorted(vin_df.columns)

['ABS',
 'AEB',
 'AdaptiveCruiseControl',
 'AdaptiveHeadlights',
 'AdditionalErrorText',
 'AirBagLocCurtain',
 'AirBagLocFront',
 'AirBagLocKnee',
 'AirBagLocSeatCushion',
 'AirBagLocSide',
 'Artemis',
 'AxleConfiguration',
 'Axles',
 'BasePrice',
 'BatteryA',
 'BatteryCells',
 'BatteryInfo',
 'BatteryKWh',
 'BatteryModules',
 'BatteryPacks',
 'BatteryType',
 'BatteryV',
 'BedLengthIN',
 'BedType',
 'BlindSpotMon',
 'BodyCabType',
 'BodyClass',
 'BrakeSystemDesc',
 'BrakeSystemType',
 'BusFloorConfigType',
 'BusLength',
 'BusType',
 'CAFEBodyType',
 'CAFEMake',
 'CAFEModel',
 'CashForClunkers',
 'ChargerLevel',
 'ChargerPowerKW',
 'CoolingType',
 'Country',
 'CurbWeightLB',
 'CustomMotorcycleType',
 'DestinationMarket',
 'DisplacementCC',
 'DisplacementCI',
 'DisplacementL',
 'Doors',
 'DriveType',
 'DriverAssist',
 'ESC',
 'EVDriveUnit',
 'ElectrificationLevel',
 'EngineConfiguration',
 'EngineCycles',
 'EngineCylinders',
 'EngineHP',
 'EngineKW',
 'EngineManufacturer',
 'EngineModel'

In [191]:
all_dfs.keys()

['matched_vins_ranges',
 'matched_vins_no_dupes',
 'ignore_vins',
 'vin',
 'matched_vins_simple',
 'epa']

In [193]:
all_dfs['matched_vins_simple'].head()

,matched_on,year_vin,Series_vin,fuelType1_vin,VIN_vin,drive_vin,Trim2_vin,make_vin,transmission_speeds_vin,BodyClass_vin,...,EPA_epa,transmission_speeds_mod_epa,transmission_speeds_epa,transmission_type_mod_epa,transmission_type_epa,displ_mod_epa,vtyp3_epa,model_mod_epa,EPA_ID_epa,type_epa
0,"make, model_mod, year, fuelType1_mod",1995,-1,gasoline,1b3ej46cxsn599296,-1,-1,dodge,-1,sedan/saloon,...,1955,4,4,auto,auto,2.4,-1,stratus,2165,0
1,"make, model_mod, year, fuelType1_mod",1995,-1,gasoline,1b3ej46cxsn599296,-1,-1,dodge,-1,sedan/saloon,...,1956,4,4,auto,auto,2.5,-1,stratus,2166,0
2,"make, model_mod, year, fuelType1_mod",1995,-1,gasoline,1b3ej46cxsn652272,-1,-1,dodge,-1,sedan/saloon,...,1955,4,4,auto,auto,2.4,-1,stratus,2165,0
3,"make, model_mod, year, fuelType1_mod",1995,-1,gasoline,1b3ej46cxsn652272,-1,-1,dodge,-1,sedan/saloon,...,1956,4,4,auto,auto,2.5,-1,stratus,2166,0
4,"make, model_mod, year, fuelType1_mod",1995,-1,gasoline,1b3ej56cxsn596520,-1,-1,dodge,-1,sedan/saloon,...,1955,4,4,auto,auto,2.4,-1,stratus,2165,0


In [196]:
sorted(ordered_cols, reverse=False
    )

['BodyClass_vin',
 'EPA_ID_epa',
 'GVWR_vin',
 'Series_vin',
 'VClass_epa',
 'VIN_ID_vin',
 'VIN_vin',
 'VehicleType_vin',
 'city08_epa',
 'comb08_epa',
 'counts_vin',
 'cylinders_epa',
 'cylinders_vin',
 'displ_epa',
 'displ_mod_epa',
 'displ_mod_vin',
 'displ_vin',
 'drive_epa',
 'drive_mod_epa',
 'drive_mod_vin',
 'drive_vin',
 'error_id_vin',
 'fuelType1_epa',
 'fuelType1_mod_epa',
 'fuelType1_mod_vin',
 'fuelType1_vin',
 'highway08_epa',
 'make_epa',
 'make_vin',
 'matched_on',
 'model_epa',
 'model_mod_epa',
 'model_mod_vin',
 'model_vin',
 'transmission_speeds_epa',
 'transmission_speeds_mod_epa',
 'transmission_speeds_mod_vin',
 'transmission_speeds_vin',
 'transmission_type_epa',
 'transmission_type_mod_epa',
 'transmission_type_mod_vin',
 'transmission_type_vin',
 'trany_epa',
 'type_epa',
 'type_from_vin_vin',
 'type_vin',
 'vtyp3_vin',
 'weight_vin',
 'year_epa',
 'year_vin']